### 1.3 Unbreakable encryption
---

A one-time pad is way of encryption a piece of data by combining it with meaningless random dummy data in such a way that the original cannot be reconstituted without access to both the product and the dummy data. In essence, this leaves the encryter with a key pair. One key is the product, and the other is the random dummy data. One key on its own is useless; only the combination of both keys can unlock the origical data. When performed correctly, a one-time pad is a form of unbreakable encrytion. 

#### 1.3.1 Getting the data in order

In this example, we will encrypt a str using a one-time pad. One way thinking about a Python 3 str is as a sequence of UTF-8 bytes (with UTF-8 using a Unicode character encoding). A str is as a sequence of UTF-8 bytes (represented as the bytes type) throught the encode() method. Likewise, a sequence of UTF-8 bytes can be converted back into a str using the decode() method on the bytes type.

There are three criteria that the dummy data used in a one-time pad encryption operation must meet for the resulting product to be unbreakable. The dummy data must be the same length as the original data, truly random, and completely secret. The first and third criteria are common sense. If the dummy data repeats because it it too short, there could be an observed pattern. If one of the keys is not truly secret (perhaps it is reused elsewhere or partially revealed), then an attacker has a clue. The second criteria poses a question all its own; how to produce truly random data.

In this example we will use the pseudo-random data generating function token_bytes() from the secrets module (first included in the standard library in Python) Our data will not be truly random, in the sense that the secrets package still is using a pseudo-random number generator behind the scenes, but it will be close enough for our purposes. Let's generate a random key for use as dummy data.


In [1]:
from secrets import token_bytes
from typing import Tuple

def random_key(length: int) -> int:
    # generate length random bytes
    tb: bytes = token_bytes(length)
    # convert those bytes into a bit string and return it
    return int.from_bytes(tb, "big")

This function creates an int filled with length random bytes. The method int.from_bytes() is used to convert from bytes to int. How can multiple bytes be converted to a single integer? The answer lies in section 1.2. In that section, you learned that the int type can be of arbitrary size, and you saw how it can be used as a generic bit string. int is being used in the same way here. For example, the from_bytes() method will take 7 bytes (7bytes * 8bits = 56bits) and convert them into a 56-bit integer. Bitwise operations can be executed more easily and performantly on a single int (read "long bit string") than on many individual bytes in a sequence, this sequence uses the Bitwise operation XOR. 

#### 1.3.2 Encrypting and decrypting

How will the dummy data be combined with the original data that we want to encrypt? The XOR operation will serve this purpose. XOR is a logical bitwise (operates at the bit level) operation that returns true when one of its operands is true but returns false when both are true or niether is true. As you may have guessed, XOR stands for *exclusive or*.

In Python, the XOR operator is ^. In the contect of the bits of binary numbers, XOR returns 1 for 0^1, but 0 for 0^0 and 1^1. If the bits of two numbers are combined using XOR, a helpful property is that the product can be recombined with either of the operands to produce the other operand:

A ^ B = C
C ^ B = A
C ^ A = B

This key insight forms the basis of one-time pad encryption. To form our product, we will simply XOR an int representing the bytes in our original str with a randomly generated int of the same bit length (as produced by random_key()) . Our returned key pair will be the dummy data and the product. 

In [2]:
def encrypt(original: str) -> Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy: int = random_key(len(original_bytes))
    original_key: int = int.from_bytes(original_bytes, "big")
    encrypted: int = original_key ^ dummy # XOR
    return dummy, encrypted

**NOTE** int.from_bytes() is being passed two arguements. The first is the bytes that we want to convert into an int. The second is the *endianness* of those bytes ("big"). Endianness refers to the byte-ordering used to store data. Does the most significant byte come first, or does the least significant byte come first? In this case, it does not matter as long as we use the same ordering both when we encrypt and decrypt, because we are actually only manipulating the data at the individual bit leve. In other situations, when you are not controlling both ends of the encoding process, the ordering can absolutely matter.

Decryption is simply a matter of recombining the key pair we generated with encrypt(). This is achieved once again by doing an XOR operation between each and every bit in the two keys. The ultimate output must be converted back to a str. First, the int is converted to byters using int.to_bytes(). This method requires the number of bytes to be converted from the int. To get this number, we divide the bit length by eight (the number of bits in a byte). Finally, the bytes method decode() gives us back a str.

In [3]:
def decrypt(key1: int, key2: int) -> str:
    decrypted: int = key1 ^ key2 # XOR
    temp: bytes = decrypted.to_bytes((decrypted.bit_length() + 7) // 8, "big")
    return temp.decode()

It was neccessary to add 7 to the length of the decrypted data before using integer-division (//) to divide by 8 to ensure that we "round up", to avoid an off-by-one error. If our one time pad encryption truly works, we should be able to encrypt and decrypt the same Unicode string without issue.

In [ ]:
if __name__ == "__main__":
    key1, key2 = encrypt("One Time Pad!")